In [39]:
import os
import re

from estnltk import EnvelopingBaseSpan
from estnltk import Text, Layer, Annotation, EnvelopingSpan, Span
from estnltk.converters import json_to_text
from estnltk.layer_operations import extract_sections

In [78]:
file = "Harju_Hageri_Kohila_id4177_1883a"
#
#Harju_Keila_Saue_id13436_1885a
with open("vallakohus_esimene/" + str(file) + ".txt", "r", encoding="utf-8") as txt:
    text = txt.read()
with open("vallakohtufailid_json/" + str(file) + ".json", "r", encoding="utf-8") as json:
    imported = json_to_text(json.read())

In [79]:
indexes_on_line_split = re.compile(r' (\d+) (\d+;\d+ ){1,}(\d+)$')

def collect_annotations( in_f ):
    annotations = []
    split_lines_ahead = 0
    for line in in_f:
        line = line.rstrip('\n')
        items = line.split('\t')
        if split_lines_ahead > 0:
            split_lines_ahead -= 1
            last_item = annotations[-1]
            new_tuple = (last_item[0],last_item[1],last_item[2],(last_item[3]+line),last_item[4])
            annotations[-1] = new_tuple
            continue
        if len(items) == 3:
            indexes_str = items[1]
            if indexes_str.count(';') > 0:
                split_lines_ahead += indexes_str.count(';')
            indexes_str = indexes_on_line_split.sub(' \\1 \\3', indexes_str)
            tag, start, end = indexes_str.split()
            annotations.append( (tag, start, end, re.sub(r'\s\s+', r' ', items[2]).strip(), items[0]) )
    seen = set()
    removed_duplicates_annotations = []
    for a, b, c, d, e in annotations:
        if not b in seen:
            seen.add(b)
            removed_duplicates_annotations.append((a, b, c, d, e))
        else:
            for index, item in enumerate(removed_duplicates_annotations):
                if item[1] == b and item[2] > c:
                    tuple_without_n = (a, b, c, d, e)
                    item = tuple_without_n
                    removed_duplicates_annotations[index] = item
                elif item[1] == b and item[2] < c:
                    tuple_without_n = (a, b, item[2], d, e)
                    item = tuple_without_n
                    removed_duplicates_annotations[index] = item
                else:
                    continue
    
    for index, item in enumerate(removed_duplicates_annotations):
        if "\xa0" in item[3]:
            replaced = re.sub(r'\s\s+', r' ', item[3].replace(u'\xa0', u' '))
            print("replaced:",replaced)
            removed_duplicates_annotations[index] = ( item[0], item[1], item[2], replaced, item[4] )
        
    
    annotations = sorted(list(set(removed_duplicates_annotations)), key=lambda x: int(x[1]))
    return annotations

with open("vallakohus_esimene/" + str(file) + ".ann", "r", encoding="utf-8") as ann:
    ann = collect_annotations(ann)
    for annotation in ann:
        print(annotation)

('KO_koht', '17', '30', 'Jerlepi Walla', 'T1')
('Isik', '36', '47', 'Pridik Post', 'T2')
('KO_koht', '110', '129', 'Angerja mäe kõrtsis', 'T3')
('KO_koht', '232', '249', 'Angerjamäe kõrtsi', 'T4')
('Koht', '717', '726', 'Mülistele', 'T5')
('KO_koht', '834', '851', 'Angerjamäe kõrtsi', 'T6')
('KO_koht', '1026', '1042', 'Salutaguse Walla', 'T7')
('Isik', '1050', '1070', 'Herman Pritz Elepson', 'T8')
('KO_koht', '1072', '1081', 'Wanakubja', 'T9')
('Isik', '1089', '1105', 'Kustas Ristikiwi', 'T10')
('KO_koht', '1107', '1112', 'Nõmme', 'T11')
('Isik', '1113', '1125', 'Mart Kändler', 'T12')
('Isik', '1129', '1141', 'Jüri Koosler', 'T13')
('KO_koht', '1161', '1171', 'Salutaguse', 'T14')
('Org', '1248', '1270', 'Salutaguse Walla kohtu', 'T15')
('Koht', '1327', '1334', 'Angerja', 'T16')
('Isik', '1344', '1357', 'Jakob Normann', 'T17')
('Isik', '1364', '1377', 'Pritz Elepson', 'T18')
('Isik', '1379', '1395', 'Gustas Ristikivi', 'T19')
('Isik', '1397', '1409', 'Jüri Koosler', 'T20')
('Isik', '141

In [80]:
imported.gold_ner

Layer(name='gold_ner', attributes=('nertag',), spans=SL[Span(['Jerlepi', 'Walla'], [{'nertag': 'LOC_ORG'}]),
Span(['Pridik', 'Post-'], [{'nertag': 'PER'}]),
Span(['Angerja', 'mäe', 'kõrtsis'], [{'nertag': 'LOC_ORG'}]),
Span(['Angerjamäe', 'kõrtsi'], [{'nertag': 'LOC_ORG'}]),
Span(['Mülistele'], [{'nertag': 'LOC'}]),
Span(['Angerjamäe', 'kõrtsi'], [{'nertag': 'LOC_ORG'}]),
Span(['Salutaguse', 'Walla'], [{'nertag': 'LOC_ORG'}]),
Span(['Herman', 'Pritz', 'Elepson'], [{'nertag': 'PER'}]),
Span(['Wanakubja'], [{'nertag': 'LOC_ORG'}]),
Span(['Kustas', 'Ristikiwi'], [{'nertag': 'PER'}]),
Span(['Nõmme'], [{'nertag': 'LOC_ORG'}]),
Span(['Mart', 'Kändler'], [{'nertag': 'PER'}]),
Span(['Jüri', 'Koosler'], [{'nertag': 'PER'}]),
Span(['Salutaguse'], [{'nertag': 'LOC_ORG'}]),
Span(['Salutaguse', 'Walla', 'kohtu'], [{'nertag': 'ORG'}]),
Span(['Angerja'], [{'nertag': 'LOC'}]),
Span(['Jakob', 'Normann'], [{'nertag': 'PER'}]),
Span(['Pritz', 'Elepson'], [{'nertag': 'PER'}]),
Span(['Gustas', 'Ristikivi'], [{'nertag': 'PER'}]),
Span(['Jüri', 'Koosler'], [{'nertag': 'PER'}]),
Span(['Mart', 'Kändler'], [{'nertag': 'PER'}]),
Span(['Jerlepi'], [{'nertag': 'LOC_ORG'}]),
Span(['Jerlepa'], [{'nertag': 'LOC_ORG'}]),
Span(['Pritz', 'Elepshon'], [{'nertag': 'PER'}]),
Span(['Kustas', 'Ristikiwi'], [{'nertag': 'PER'}]),
Span(['Jüri', 'Koosler'], [{'nertag': 'PER'}]),
Span(['Mart', 'Kändler'], [{'nertag': 'PER'}]),
Span(['Angerjamäe', 'kõrtsis'], [{'nertag': 'LOC_ORG'}]),
Span(['Kustas', 'Ristikiwi'], [{'nertag': 'PER'}]),
Span(['Kustas', 'Ristikiwi'], [{'nertag': 'PER'}]),
Span(['Angerjamäe', 'kõrtsi'], [{'nertag': 'LOC_ORG'}]),
Span(['Jerlepimees'], [{'nertag': 'MISC'}]),
Span(['Mart', 'Kändler'], [{'nertag': 'PER'}]),
Span(['Angerjamäe', 'kõrtsi'], [{'nertag': 'LOC_ORG'}]),
Span(['Kustas', 'Ristikiwi'], [{'nertag': 'PER'}]),
Span(['Pritz', 'Elepshon'], [{'nertag': 'PER'}]),
Span(['Jüri', 'Koosler'], [{'nertag': 'PER'}]),
Span(['Kustas', 'Ristikiwi'], [{'nertag': 'PER'}]),
Span(['Jerlepimeest'], [{'nertag': 'MISC'}]),
Span(['Mihkel', 'Waartberg'], [{'nertag': 'PER'}]),
Span(['Jüri', 'Tudeberg'], [{'nertag': 'PER'}]),
Span(['Jaan', 'Wanaweski'], [{'nertag': 'PER'}]),
Span(['G', '.', 'Wender'], [{'nertag': 'PER'}])])

In [77]:
imported.gold_wordner

Layer(name='gold_wordner', attributes=('nertag',), spans=SL[Span('Mustwee', [{'nertag': 'B-LOC_ORG'}]),
Span('pääkohtumehe', [{'nertag': 'O'}]),
Span('Tristan', [{'nertag': 'B-PER'}]),
Span('Kulikowi', [{'nertag': 'I-PER'}]),
Span('weksli', [{'nertag': 'O'}]),
Span('nõudmise', [{'nertag': 'O'}]),
Span('asjas', [{'nertag': 'O'}]),
Span('wasta', [{'nertag': 'O'}]),
Span('ärasaranud', [{'nertag': 'O'}]),
Span('Kaupaaküla', [{'nertag': 'B-LOC_ORG'}]),
Span('lahtist', [{'nertag': 'O'}]),
Span('meest', [{'nertag': 'O'}]),
Span('Aleksandr', [{'nertag': 'B-PER'}]),
Span('Timofei', [{'nertag': 'I-PER'}]),
Span('Tomkina', [{'nertag': 'I-PER'}]),
Span('ja', [{'nertag': 'O'}]),
Span('nimest', [{'nertag': 'O'}]),
Span('wiimase', [{'nertag': 'O'}]),
Span('järele', [{'nertag': 'O'}]),
Span('jäänud', [{'nertag': 'O'}]),
Span('waranduse', [{'nertag': 'O'}]),
Span('pärast', [{'nertag': 'O'}]),
Span(',', [{'nertag': 'O'}]),
Span('et', [{'nertag': 'O'}]),
Span('kuidas', [{'nertag': 'O'}]),
Span('Tristan', [{'nertag': 'B-PER'}]),
Span('Kulikow', [{'nertag': 'I-PER'}]),
Span('käsiniku', [{'nertag': 'O'}]),
Span('I', [{'nertag': 'B-ORG'}]),
Span('Tartu', [{'nertag': 'I-ORG'}]),
Span('Kihelkonna', [{'nertag': 'I-ORG'}]),
Span('kohtu', [{'nertag': 'I-ORG'}]),
Span('ülesaanud', [{'nertag': 'O'}]),
Span(',', [{'nertag': 'O'}]),
Span('on', [{'nertag': 'O'}]),
Span('see', [{'nertag': 'O'}]),
Span('maja', [{'nertag': 'O'}]),
Span(',', [{'nertag': 'O'}]),
Span('kus', [{'nertag': 'O'}]),
Span('Aleksander', [{'nertag': 'B-PER'}]),
Span('Tomkin', [{'nertag': 'I-PER'}]),
Span('Kasepääl', [{'nertag': 'B-LOC'}]),
Span('elanud', [{'nertag': 'O'}]),
Span('Tomkin', [{'nertag': 'B-PER'}]),
Span('oma', [{'nertag': 'O'}]),
Span('saiwad', [{'nertag': 'O'}]),
Span('ettekutsutud', [{'nertag': 'O'}]),
Span('Kasepää', [{'nertag': 'B-LOC_ORG'}]),
Span('küla', [{'nertag': 'I-LOC_ORG'}]),
Span('wanad', [{'nertag': 'O'}]),
Span('majaperemehed', [{'nertag': 'O'}]),
Span('Jakow', [{'nertag': 'B-PER'}]),
Span('Mihailow', [{'nertag': 'I-PER'}]),
Span('Annusohkin', [{'nertag': 'B-PER'}]),
Span(',', [{'nertag': 'O'}]),
Span('Tartu', [{'nertag': 'B-LOC_ORG'}]),
Span('lina', [{'nertag': 'I-LOC_ORG'}]),
Span('akerdist', [{'nertag': 'O'}]),
Span('Awdei', [{'nertag': 'B-PER'}]),
Span('Iwanow', [{'nertag': 'I-PER'}]),
Span('Amtsewnikow', [{'nertag': 'I-PER'}]),
Span(',', [{'nertag': 'O'}]),
Span('Tartu-', [{'nertag': 'B-LOC_ORG'}]),
Span('lina', [{'nertag': 'I-LOC_ORG'}]),
Span('oksadik', [{'nertag': 'O'}]),
Span('Iwan', [{'nertag': 'B-PER'}]),
Span('Petrow', [{'nertag': 'I-PER'}]),
Span('Sideorw', [{'nertag': 'I-PER'}]),
Span(',', [{'nertag': 'O'}]),
Span('Tartu', [{'nertag': 'B-LOC_ORG'}]),
Span('-', [{'nertag': 'I-LOC_ORG'}]),
Span('lina', [{'nertag': 'I-LOC_ORG'}]),
Span('okeadit', [{'nertag': 'O'}]),
Span('ja', [{'nertag': 'O'}]),
Span('Kasepää', [{'nertag': 'B-LOC_ORG'}]),
Span('sotnik', [{'nertag': 'O'}]),
Span('Allatskiwwi', [{'nertag': 'B-LOC_ORG'}]),
Span('wallamees', [{'nertag': 'O'}]),
Span('Stepan', [{'nertag': 'B-PER'}]),
Span('Jakowlew', [{'nertag': 'I-PER'}]),
Span('Säichtschikow', [{'nertag': 'I-PER'}]),
Span('ja', [{'nertag': 'O'}]),
Span('tunnistasiwad', [{'nertag': 'O'}]),
Span('nemad', [{'nertag': 'O'}]),
Span('ja', [{'nertag': 'O'}]),
Span('wõiwad', [{'nertag': 'O'}]),
Span('oma', [{'nertag': 'O'}]),
Span('tunnistust', [{'nertag': 'O'}]),
Span('wandega', [{'nertag': 'O'}]),
Span('kinnitada', [{'nertag': 'O'}]),
Span(',', [{'nertag': 'O'}]),
Span('et', [{'nertag': 'O'}]),
Span('see', [{'nertag': 'O'}]),
Span('maja', [{'nertag': 'O'}]),
Span(',', [{'nertag': 'O'}]),
Span('kus', [{'nertag': 'O'}]),
Span('sees', [{'nertag': 'O'}]),
Span('Aleksander', [{'nertag': 'B-PER'}]),
Span('Timofe', [{'nertag': 'I-PER'}]),
Span('Tomkin', [{'nertag': 'I-PER'}]),
Span('elanud', [{'nertag': 'O'}]),
Span(',', [{'nertag': 'O'}]),
Span('olla', [{'nertag': 'O'}]),
Span('tema', [{'nertag': 'O'}]),
Span('äija', [{'nertag': 'O'}]),
Span('Tartu-lina', [{'nertag': 'B-LOC_ORG'}]),
Span('okevidi'